In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pickle
import geopandas as gpd
import numpy as np
from pykrige.rk import Krige
import matplotlib.pyplot as plt

---

In [2]:
res  =pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/outputs/sample_a/regression_fine_tuning.csv', index_col=0)
for params, df in res.groupby(['param_max_depth', 'param_min_samples_leaf']):
    print(params, '\n', df[['mean_test_score', 'std_test_score']].mean(), '\n')

(5, 1) 
 mean_test_score    0.390024
std_test_score     0.049714
dtype: float64 

(5, 5) 
 mean_test_score    0.391251
std_test_score     0.049858
dtype: float64 

(5, 10) 
 mean_test_score    0.390355
std_test_score     0.050246
dtype: float64 

(50, 1) 
 mean_test_score    0.436570
std_test_score     0.050115
dtype: float64 

(50, 5) 
 mean_test_score    0.438116
std_test_score     0.049690
dtype: float64 

(50, 10) 
 mean_test_score    0.433808
std_test_score     0.049627
dtype: float64 

(100, 1) 
 mean_test_score    0.436570
std_test_score     0.050115
dtype: float64 

(100, 5) 
 mean_test_score    0.438116
std_test_score     0.049690
dtype: float64 

(100, 10) 
 mean_test_score    0.433808
std_test_score     0.049627
dtype: float64 



In [3]:
#### THE FINAL CHOICE IS: max_depth': 50, 'min_samples_leaf': 5

rs = 516

X_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/final_X_train_val.csv')
y_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/y_train_val.csv')

regression_forest = RandomForestRegressor(max_depth=50,
                                          min_samples_leaf=5,
                                          n_jobs=-1, 
                                          random_state=rs
                                          )


regression_forest.fit(X_train, y_train)

with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/trained_models/final_regression.pkl', 'wb') as f:
    pickle.dump(regression_forest, f)

/nfs/home/genovese/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [4]:
with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/trained_models/final_regression.pkl', 'rb') as f:
    model = pickle.load(f)

In [5]:
y_train_pred = model.predict(X_train).round(0)
print(r2_score(y_train_pred, y_train['target']))
print(accuracy_score(pd.Series(y_train_pred), pd.Series(y_train['target'])))

X_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/final_X_test.csv')
y_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/y_test.csv')

y_test_pred = model.predict(X_test).round(0)
print(r2_score(y_test_pred, y_test['target']))
print(accuracy_score(pd.Series(y_test_pred), pd.Series(y_test['target'])))

0.6060891472437637
0.23246102449888642
-0.30240075830748303
0.12145110410094637


In [6]:
def predict_dispersion(crosstab):
    total = 0
    for i, row in enumerate(crosstab.to_numpy()):
        for j, val in enumerate(row):
            total += val * abs(j - i)
    return total/(crosstab.sum().sum())*(crosstab.shape[1])

In [7]:
train_mat= pd.crosstab(y_train.target, y_train_pred)
test_mat = pd.crosstab(y_test.target, y_test_pred)

In [8]:
test_mat.loc[:, 0.0] = 0
test_mat = test_mat.loc[:, [0,1,2,3,4,5,6,7,8,9,10]]

In [9]:
print(predict_dispersion(train_mat))
print(predict_dispersion(test_mat))

14.956570155902005
25.08832807570978


---

In [10]:
res  =pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/outputs/sample_a/classification_fine_tuning.csv', index_col=0)
for params, df in res.groupby(['param_max_depth', 'param_min_samples_leaf']):
    print(params, '\n', df[['mean_test_score', 'std_test_score']].mean(), '\n')

(5, 1) 
 mean_test_score    0.426790
std_test_score     0.018331
dtype: float64 

(5, 5) 
 mean_test_score    0.425121
std_test_score     0.018470
dtype: float64 

(5, 10) 
 mean_test_score    0.427346
std_test_score     0.015664
dtype: float64 

(50, 1) 
 mean_test_score    0.427068
std_test_score     0.019152
dtype: float64 

(50, 5) 
 mean_test_score    0.425672
std_test_score     0.020209
dtype: float64 

(50, 10) 
 mean_test_score    0.433466
std_test_score     0.016536
dtype: float64 

(100, 1) 
 mean_test_score    0.427068
std_test_score     0.019152
dtype: float64 

(100, 5) 
 mean_test_score    0.425672
std_test_score     0.020209
dtype: float64 

(100, 10) 
 mean_test_score    0.433466
std_test_score     0.016536
dtype: float64 



In [11]:
#### THE FINAL CHOICE IS: max_depth': 50, 'min_samples_leaf': 10

rs = 516

X_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/final_X_train_val.csv')
y_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/y_train_val.csv')

classification_forest = RandomForestClassifier(max_depth=50,
                                          min_samples_leaf=10,
                                          n_jobs=-1, 
                                          random_state=rs
                                          )


classification_forest.fit(X_train, y_train)

with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/trained_models/final_classification.pkl', 'wb') as f:
    pickle.dump(classification_forest, f)

/nfs/home/genovese/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [12]:
with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/trained_models/final_classification.pkl', 'rb') as f:
    model = pickle.load(f)

In [13]:
y_train_pred = model.predict(X_train).round(0)
print(r2_score(y_train_pred, y_train['target']))
print(accuracy_score(pd.Series(y_train_pred), pd.Series(y_train['target'])))

X_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/final_X_test.csv')
y_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_a/y_test.csv')

y_test_pred = model.predict(X_test).round(0)
print(r2_score(y_test_pred, y_test['target']))
print(accuracy_score(pd.Series(y_test_pred), pd.Series(y_test['target'])))

0.4809758138981731
0.5645879732739421
0.2911914872688989
0.444794952681388


In [14]:
train_mat= pd.crosstab(y_train.target, y_train_pred)
test_mat = pd.crosstab(y_test.target, y_test_pred)

In [15]:
train_mat.loc[:, [8.0, 9.0]]=0
train_mat = train_mat.loc[:, [0,1,2,3,4,5,6,7,8,9,10]]

In [16]:
print(predict_dispersion(train_mat))
print(predict_dispersion(test_mat))

20.68318485523385
13.905362776025237


---